In [14]:
from pymodbus.client import ModbusTcpClient

# --- Parámetros de Conexión (Modifica según tu configuración) ---
# IP del conversor USR-TCP232-304 en tu red LAN
CONVERTER_IP = '192.168.0.21'
# Puerto TCP configurado en el conversor (Modo TCP Server)
CONVERTER_PORT = 23
# Dirección del esclavo (valor del parámetro P0308 en el WEG)
WEG_SLAVE_ID = 1

# --- Parámetro a Leer (Ver NOTA IMPORTANTE abajo) ---
# Este es un EJEMPLO. Debes verificar el registro correcto en el manual Modbus del WEG.
# El registro para P0002 (Velocidad de Salida) suele ser 2.
REGISTER_ADDRESS = 2
# Cantidad de registros a leer
QUANTITY = 1

In [15]:
# Inicializa el cliente Modbus TCP
client = ModbusTcpClient(CONVERTER_IP, port=CONVERTER_PORT)

try:
    print(f"Intentando conectar a {CONVERTER_IP}:{CONVERTER_PORT}...")
    # Conecta al conversor
    client.connect()

    print(f"Leyendo registro {REGISTER_ADDRESS} del esclavo {WEG_SLAVE_ID}...")
    # Realiza la lectura del registro (función 0x03 de Modbus)
    response = client.read_holding_registers(
        address=REGISTER_ADDRESS,
        count=QUANTITY,
    )

    # Comprueba si hubo un error en la respuesta
    if response.isError():
        print(f"Error al leer del dispositivo: {response}")
    else:
        # Si la lectura fue exitosa, muestra el valor
        valor_registro = response.registers[0]
        print(f"¡Éxito! El valor del registro {REGISTER_ADDRESS} es: {valor_registro}")
        # Nota: El valor puede necesitar ser escalado. Por ejemplo, si la escala es 1 decimal,
        # un valor de 600 podría significar 60.0 Hz.
        print(f"(Recuerda verificar la escala del parámetro en el manual del WEG)")

except Exception as e:
    print(f"No se pudo conectar al dispositivo. Error: {e}")

finally:
    # Cierra la conexión
    client.close()
    print("Conexión cerrada.")

Intentando conectar a 192.168.0.21:23...
Leyendo registro 2 del esclavo 1...


No response received after 3 retries, continue with next request


No se pudo conectar al dispositivo. Error: Modbus Error: [Input/Output] No response received after 3 retries, continue with next request
Conexión cerrada.


In [29]:
from pymodbus.client import ModbusTcpClient

# Dirección IP del USR y puerto que configuraste
client = ModbusTcpClient('192.168.0.21', port=502)
client.connect()

# Leer 1 registro empezando en la dirección 1 del esclavo 1
rr = client.read_holding_registers(address=1,count=2,device_id=1)

if not rr.isError():
    print(f"Valor leído: {rr.registers[0]}")
else:
    print("Error al leer")

client.close()


No response received after 3 retries, continue with next request


ModbusIOException: Modbus Error: [Input/Output] No response received after 3 retries, continue with next request

In [31]:
from pymodbus.client import ModbusTcpClient

client = ModbusTcpClient(host='192.168.0.21', port=502)       # Create client object
client.connect()                               # connect to device
client.write_coil(1, True, device_id=1)        # set information in device
result = client.read_coils(2, 3, device_id=1)  # get information from device
print(result.bits[0])                          # use information
client.close()                                 # Disconnect device

No response received after 3 retries, continue with next request


ModbusIOException: Modbus Error: [Input/Output] No response received after 3 retries, continue with next request

In [32]:
"""
Comunicación simple con Variador WEG CFW500 usando PyModbus
Requiere: pip install pymodbus
"""

from pymodbus.client import ModbusTcpClient
from pymodbus.exceptions import ModbusException
import time

class WEGVariadorPyModbus:
    """Clase simplificada usando PyModbus para WEG CFW500"""
    
    def __init__(self, ip_address: str, port: int = 502, unit_id: int = 1):
        self.ip_address = ip_address
        self.port = port
        self.unit_id = unit_id
        self.client = None
    
    def connect(self) -> bool:
        """Conecta al variador via TCP"""
        try:
            self.client = ModbusTcpClient(self.ip_address, port=self.port)
            if self.client.connect():
                print(f"Conectado a {self.ip_address}:{self.port}")
                return True
            else:
                print("Error al conectar")
                return False
        except Exception as e:
            print(f"Error de conexión: {e}")
            return False
    
    def disconnect(self):
        """Desconecta del variador"""
        if self.client:
            self.client.close()
            print("Desconectado")
    
    def read_parameter(self, parameter_number: int):
        """Lee un parámetro del variador (P0001, P0002, etc.)"""
        try:
            # Los parámetros P0xxx se mapean a direcciones Modbus
            # P0001 -> dirección 0x0001, P0002 -> dirección 0x0002, etc.
            result = self.client.read_holding_registers(parameter_number, 1, unit=self.unit_id)
            
            if result.isError():
                print(f"Error leyendo P{parameter_number:04d}: {result}")
                return None
            else:
                value = result.registers[0]
                print(f"P{parameter_number:04d} = {value}")
                return value
                
        except Exception as e:
            print(f"Excepción leyendo P{parameter_number:04d}: {e}")
            return None
    
    def write_parameter(self, parameter_number: int, value: int) -> bool:
        """Escribe un parámetro en el variador"""
        try:
            result = self.client.write_register(parameter_number, value, unit=self.unit_id)
            
            if result.isError():
                print(f"Error escribiendo P{parameter_number:04d}: {result}")
                return False
            else:
                print(f"P{parameter_number:04d} = {value} (escrito)")
                return True
                
        except Exception as e:
            print(f"Excepción escribiendo P{parameter_number:04d}: {e}")
            return False
    
    def get_frequency(self):
        """Lee frecuencia de referencia P0002"""
        value = self.read_parameter(2)  # P0002
        if value is not None:
            return value / 100.0  # Convertir de centésimas de Hz a Hz
        return None
    
    def set_frequency(self, frequency_hz: float) -> bool:
        """Establece frecuencia de referencia P0002"""
        value = int(frequency_hz * 100)  # Convertir a centésimas de Hz
        return self.write_parameter(2, value)
    
    def get_slave_address(self):
        """Lee la dirección del esclavo P0001"""
        return self.read_parameter(1)  # P0001
    
    def start_motor(self) -> bool:
        """Comando de arranque P0003 = 1"""
        return self.write_parameter(3, 1)  # P0003 = 1
    
    def stop_motor(self) -> bool:
        """Comando de parada P0003 = 0"""
        return self.write_parameter(3, 0)  # P0003 = 0

def main():
    """Función principal de prueba"""
    print("=== Comunicación WEG CFW500 con PyModbus ===")
    
    # Configuración
    IP = "192.168.0.21"
    PORT = 502
    UNIT_ID = 1  # Dirección Modbus del variador
    
    # Crear instancia
    variador = WEGVariadorPyModbus(IP, PORT, UNIT_ID)
    
    try:
        # Conectar
        if not variador.connect():
            return
        
        print("\n--- Leyendo parámetros básicos ---")
        
        # Verificar dirección del esclavo
        slave_addr = variador.get_slave_address()
        if slave_addr != UNIT_ID:
            print(f"ADVERTENCIA: Dirección esperada {UNIT_ID}, encontrada {slave_addr}")
        
        # Leer frecuencia actual
        freq = variador.get_frequency()
        if freq is not None:
            print(f"Frecuencia actual: {freq} Hz")
        
        # Leer algunos otros parámetros comunes
        # P0004 - Estado del variador
        status = variador.read_parameter(4)
        
        # P0005 - Corriente de salida
        current_raw = variador.read_parameter(5)
        if current_raw is not None:
            current = current_raw / 100.0
            print(f"Corriente de salida: {current} A")
        
        # Ejemplo de escritura (¡CUIDADO! Solo para pruebas)
        print("\n--- Ejemplo de escritura (comentado por seguridad) ---")
        # if variador.set_frequency(25.0):
        #     print("Frecuencia establecida a 25 Hz")
        #     time.sleep(1)
        #     new_freq = variador.get_frequency()
        #     print(f"Nueva frecuencia: {new_freq} Hz")
        
    except KeyboardInterrupt:
        print("\nInterrumpido por usuario")
    
    except Exception as e:
        print(f"Error general: {e}")
    
    finally:
        variador.disconnect()

if __name__ == "__main__":
    main()

=== Comunicación WEG CFW500 con PyModbus ===
Conectado a 192.168.0.21:502

--- Leyendo parámetros básicos ---
Excepción leyendo P0001: ModbusClientMixin.read_holding_registers() got an unexpected keyword argument 'unit'
ADVERTENCIA: Dirección esperada 1, encontrada None
Excepción leyendo P0002: ModbusClientMixin.read_holding_registers() got an unexpected keyword argument 'unit'
Excepción leyendo P0004: ModbusClientMixin.read_holding_registers() got an unexpected keyword argument 'unit'
Excepción leyendo P0005: ModbusClientMixin.read_holding_registers() got an unexpected keyword argument 'unit'

--- Ejemplo de escritura (comentado por seguridad) ---
Desconectado
